In [33]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
%matplotlib inline

import umap
import umap.plot
import scipy.io as sio
import hdbscan

from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score

from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list, fcluster
from scipy.spatial.distance import pdist

from datetime import datetime
from scipy.io import savemat, loadmat
import mat73

In [34]:
data_path = '/media/user/teamshare/nick/behavior/grooming/2024-11-18-17-15-17_umap_dFF.mat'
mat_contents =  mat73.loadmat(data_path)

In [35]:
print(mat_contents)

{'data_for_UMAP': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [-0.,  0.,  0., ...,  0.,  0.,  0.],
       [-0.,  0.,  0., ...,  0.,  0.,  0.],
       ...,
       [-0., -0., -0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]], dtype=float32), 'labels_for_UMAP': [['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right'], ['Right']

In [36]:
# print(mat_contents['behavior_index'][0])

# bFiles = np.array([x[0][0] for x in mat_contents['bFilepath']])
# bIdx = np.array([x for x in mat_contents['behavior_index']])
labels = np.array([x[0] for x in mat_contents['labels_for_UMAP']])
fMat = np.array(mat_contents['data_for_UMAP'])
print(labels)
# print(bIdx.shape)
print(np.shape(fMat))

['Right' 'Right' 'Right' ... 'Elliptical Left' 'Elliptical Left'
 'Elliptical Left']
(3585, 16384)


In [41]:
# reducer = umap.UMAP(min_dist=0.1, n_components=2,n_neighbors=10,metric='correlation')
reducer = umap.UMAP(min_dist=0.0, n_components=2, n_neighbors=10, metric='cosine', random_state=42)
reducer.fit(fMat)

/home/user/anaconda3/envs/nick/lib/python3.11/site-packages/umap/umap_.py:1943: UserWarning: n_jobs value -1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(f"n_jobs value {self.n_jobs} overridden to 1 by setting random_state. Use no seed for parallelism.")


UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_neighbors=10, random_state=42, tqdm_kwds={'bar_format': '{desc}: {percentage:3.0f}%| {bar} {n_fmt}/{total_fmt} [{elapsed}]', 'desc': 'Epochs completed', 'disable': True})

In [42]:
embedding = reducer.transform(fMat)
# Verify that the result of calling transform is
# idenitical to accessing the embedding_ attribute
assert(np.all(embedding == reducer.embedding_))
embedding.shape

(3585, 2)

In [43]:
# hover_data = pd.DataFrame({'behavior':labels,
#                            'filePath':bFiles,
#                            'frameStart':bIdx[:,0],
#                            'frameEnd':bIdx[:,1]})

hover_data = pd.DataFrame({'behavior':labels})
labelMap = hover_data.behavior.replace({
    'Elliptical':'0',
    'Right':'1',
    'Left':'2',
    'Right Asymmetric':'3',
    'Left Asymmetric':'4',
    'Elliptical Right':'5',
    'Elliptical Left':'6',
    'Lick': '7',
    
})

colorKey = dict(zip(labelMap, labelMap.map(
    {
        '0':'#1f77b4',
        '1':'#2ca02c',
        '2':'#d62728',
        '3':'#9467bd',
        '4':'#8c564b',
        '5':'#ff7f0e',
        '6':'#e377c2',
        '7':'#7f7f7f',
    } 
)))

In [44]:
cols = []
for i,lab in enumerate(labels):
    if lab == "Elliptical":
        cols.append('#1f77b4')
    elif lab == "Elliptical Asymmetric":
        cols.append('#ff7f0e')
    elif lab == "Right":
        cols.append('#2ca02c')
    elif lab == "Left":
        cols.append('#d62728')
    elif lab == "Right Asymmetric":
        cols.append('#9467bd')
    elif lab == "Left Asymmetric":
        cols.append('#8c564b')
    elif lab == "Large Bilateral":
        cols.append('#e377c2')


test = np.unique(cols)

overall_silhouette = silhouette_score(embedding, labelMap)
print(overall_silhouette)

# sns.set(style='white', context='notebook', rc={'figure.figsize':(14,10)})
# fig = plt.figure()
# ax = fig.add_subplot()
# for ii, col in enumerate(np.unique(cols)):
#     idx = [True if test[ii] == i else False for i in cols]
#     ax.scatter(embedding[idx,0], embedding[idx,1], c=col, s=5)

# plt.title('UMAP projection of behavior features', fontsize=24);
# leg = plt.legend(["Elliptical", "Right", "Left Asymmetric", "Right Asymmetric", "Left", "Large Bilateral", "Elliptical Asymmetric"],prop={'size': 16})
# for legobj in leg.legend_handles:
#     legobj.set_linewidth(5)
# plt.xlabel('UMAP 1')
# plt.ylabel('UMAP 2')
# plt.gca().set_aspect('equal', 'datalim')



p = umap.plot.interactive(reducer, labels=labelMap, hover_data=hover_data, point_size=5, color_key=colorKey)
p.title.text = "Silhouette: " + str(overall_silhouette)
umap.plot.show(p)

/home/user/anaconda3/envs/nick/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/user/anaconda3/envs/nick/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


-0.14613377
